In [1]:
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"
import os
from datetime import datetime
#from datetime import date
import pymysql
from sqlalchemy import create_engine
from decouple import config 
from dotenv import load_dotenv
from pandas import to_datetime, read_sql_query
from enum import Enum



In [2]:
class Period(Enum):
    date = '2021-10-01'
    wb=''

In [3]:
load_dotenv()
# get the environment variables needed
USER = config('USRCaris')
PASSWORD = config('PASSCaris')
HOSTNAME = config('HOSTCaris')
DBNAME = config('DBCaris')

# get the engine to connect and fetch
engine = create_engine(
    f"mysql+pymysql://{USER}:{PASSWORD}@{HOSTNAME}/{DBNAME}")

query_presenceQty = f'''
SELECT 
    a.*, COUNT(*) AS qte_pres_by_creator
FROM
    (SELECT 
            dgs.created_by,
            au.email AS email,
            au.username
    FROM
        caris_db.dream_group_session dgs
    LEFT JOIN dream_group_attendance dga ON dgs.id = dga.id_group_session
    LEFT JOIN dream_group dg ON dgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dgs.created_by
    WHERE
        dga.value = 'P'
            AND dgs.date >= '{Period.date.value}') a
GROUP BY a.created_by
'''

query_beneficiariesQty = f'''
SELECT *, COUNT(*)AS qte_by_creator FROM
    (SELECT
        dgs.id AS id_session,
            topic,
            date,
            dga.id_patient AS id_patient,
            dgs.created_by,
            lc.name AS commune,
            ld.name AS departement,
            dh.name AS hub,
            au.email AS email,
            au.username,
            b.first_name,
            b.last_name,
            p.patient_code as dreams_code
    FROM
        caris_db.dream_group_session dgs
    INNER JOIN dream_group_attendance dga ON dgs.id = dga.id_group_session
        LEFT JOIN dream_group dg ON dgs.id_group = dg.id
    LEFT JOIN dream_hub dh ON dg.id_dream_hub = dh.id
    LEFT JOIN lookup_commune lc ON lc.id = dh.commune
    LEFT JOIN lookup_departement ld ON ld.id = lc.departement
    LEFT JOIN auth_users au ON au.id = dgs.created_by
    left join beneficiary b on dga.id_patient=b.id_patient
    left join patient p on p.id=dga.id_patient
    WHERE
        dga.value = 'P'
            AND dgs.date >= '{Period.date.value}') a
            group by a.id_patient
'''

pres_byAgent= read_sql_query(query_presenceQty, engine, parse_dates=True)
ben_byAgent = read_sql_query(query_beneficiariesQty, engine, parse_dates=True)


# close the pool of connection
engine.dispose()


True

In [4]:
pres_byAgent
ben_byAgent

,created_by,email,username,qte_pres_by_creator
0,929,johane.jules@carisfoundationintl.org,johanejules,16
1,1553,byronfabrice9@gmail.com,fbyron,512
2,1554,billyjolicoeur95@gmail.com,billyjolicoeur,291
3,1575,elionafredly7@gmail.com,FredlyEliona,132
4,1587,blonnichel@yahoo.fr,dchelot,912
5,1635,gerald.poteau@carisfoundationintl.org,pggd1234,532
6,1636,diegoarmando.dorlean@carisfoundationintl.org,Armando,1535
7,1637,ernst.belabre@carisfoundationintl.org,Ernstbelabre,2346
8,1679,francenelstjean.saed@yahoo.com,France14,2036
9,1883,roselord.despeignes@carisfoundationintl.org,despeignes,736


,id_session,topic,date,id_patient,created_by,commune,departement,hub,email,username,first_name,last_name,dreams_code,qte_by_creator
0,64162,1,2021-11-06,76262,1883,Verrettes,Artibonite,Ecole Mixte Benito Merilus/Desarmes,roselord.despeignes@carisfoundationintl.org,despeignes,Rose-Myrtha,Meriné,GON/DRMS/084084901,18
1,64047,1,2021-11-06,77949,1883,Verrettes,Artibonite,Institution Mixte Karl Max/Desarmes,roselord.despeignes@carisfoundationintl.org,despeignes,Love Rosenida,Dorgille,GON/DRMS/084105302,17
2,63608,1,2021-10-24,79096,1637,Dessalines,Artibonite,Collège Leopold Seghar Sengor,ernst.belabre@carisfoundationintl.org,Ernstbelabre,Loudjina,Charles,GON/DRMS/047120701,17
3,63608,1,2021-10-24,79948,1637,Dessalines,Artibonite,Collège Leopold Seghar Sengor,ernst.belabre@carisfoundationintl.org,Ernstbelabre,Lovemitatielle,Josaphat,GON/DRMS/047120703,17
4,63608,1,2021-10-24,79949,1637,Dessalines,Artibonite,Collège Leopold Seghar Sengor,ernst.belabre@carisfoundationintl.org,Ernstbelabre,Milanda,Jerome,GON/DRMS/047120704,17
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
600,64339,1,2021-11-05,129034,1910,Pétionville,Ouest,Mission Premiere Assemblee Evangelique de Dern...,twoody103@gmail.com,BonWoodRidchard,Lyzz Meredith,Lagrenade,PAP/DRMS/133348510,14
601,64339,1,2021-11-05,129035,1910,Pétionville,Ouest,Mission Premiere Assemblee Evangelique de Dern...,twoody103@gmail.com,BonWoodRidchard,Medjina,Labastille,PAP/DRMS/133348511,14
602,64339,1,2021-11-05,129036,1910,Pétionville,Ouest,Mission Premiere Assemblee Evangelique de Dern...,twoody103@gmail.com,BonWoodRidchard,Westana,Thomas,PAP/DRMS/133348512,14
603,64339,1,2021-11-05,129037,1910,Pétionville,Ouest,Mission Premiere Assemblee Evangelique de Dern...,twoody103@gmail.com,BonWoodRidchard,Myriane,Benoit,PAP/DRMS/133348513,14
